## 今天的作業
在鐵達尼資料集中，今天我們專注觀察變數之間的相關性，以Titanic_train.csv 中，首先將有遺失值的數值刪除，我們取 Titanic_train.csv 的年齡資料，試著將課堂中所學的方法應用上去。

* Q1: 產生一個新的變數(Age_above65_) Age>=65為 'Y'，其餘為'N'。  
* Q2: 添加女性和男性，產生一個新的變數(Age_above65_female)，女性或Age>=65為'Y'，其餘為'N'。  
*  Q3: 透過昨天課程的內容，驗證產生的兩個新變數，哪一個和目標變數(Survived_cate)的相關性較高?

In [1]:
# library
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
import math
import statistics
import seaborn as sns
from IPython.display import display
import sklearn
print(sklearn.__version__)
#如果只有 0.19 記得要更新至 最新版本
%matplotlib inline


#顯示所有行
pd.set_option('display.max_columns', None)
#顯示所有列
pd.set_option('display.max_rows', None)
#顯示行設定
pd.set_option('max_colwidth',100)

import pingouin as pg
import researchpy   

0.23.2


## 讀入資料

In [2]:
df_train = pd.read_csv("../data/Titanic_train.csv")
print(df_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


In [3]:
#1.產生一個新的變數 Survived_cate ，資料型態傳換成類別型態      
#2.把題目中的 Survived 用 Survived_cate 來做分析 

df_train['Survived_cate'] = df_train['Survived']
df_train['Survived_cate'] = df_train['Survived_cate'].astype('object')
print(df_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   PassengerId    891 non-null    int64  
 1   Survived       891 non-null    int64  
 2   Pclass         891 non-null    int64  
 3   Name           891 non-null    object 
 4   Sex            891 non-null    object 
 5   Age            714 non-null    float64
 6   SibSp          891 non-null    int64  
 7   Parch          891 non-null    int64  
 8   Ticket         891 non-null    object 
 9   Fare           891 non-null    float64
 10  Cabin          204 non-null    object 
 11  Embarked       889 non-null    object 
 12  Survived_cate  891 non-null    object 
dtypes: float64(2), int64(5), object(6)
memory usage: 90.6+ KB
None


In [8]:
#我們先把遺失值刪除
## 取出資料後，把遺失值刪除

complete_data = df_train[['Age', 'Survived_cate', 'Sex']].dropna()
display(complete_data.head(5))

,Age,Survived_cate,Sex
0,22.0,0,male
1,38.0,1,female
2,26.0,1,female
3,35.0,1,female
4,35.0,0,male


### Q1: 產生一個新的變數(Age_above65_) Age>=65為 'Y'，其餘為'N'。


In [9]:
complete_data['Age_above65_'] = complete_data['Age'].apply(lambda x: 'Y' if x >= 65 else 'N')
complete_data.head(10)

,Age,Survived_cate,Sex,Age_above65_
0,22.0,0,male,N
1,38.0,1,female,N
2,26.0,1,female,N
3,35.0,1,female,N
4,35.0,0,male,N
6,54.0,0,male,N
7,2.0,0,male,N
8,27.0,1,female,N
9,14.0,1,female,N
10,4.0,1,female,N


### Q2: 添加女性和男性，產生一個新的變數(Age_above65_female)，女性或Age>=65為'Y'，其餘為'N'。
* 暗示: 觀看下面影片找答案，https://www.youtube.com/watch?v=X2d-wUt5azY

In [12]:
def age_sex_map(x):
    if x.Sex == 'female' or x.Age >= 65:
        return 'Y'
    else:
        return 'N'

In [16]:
complete_data['Age_above65_female'] = complete_data[['Age', 'Sex']].apply(age_sex_map, axis=1)
complete_data.head(10)

,Age,Survived_cate,Sex,Age_above65_,Age_above65_female
0,22.0,0,male,N,N
1,38.0,1,female,N,Y
2,26.0,1,female,N,Y
3,35.0,1,female,N,Y
4,35.0,0,male,N,N
6,54.0,0,male,N,N
7,2.0,0,male,N,N
8,27.0,1,female,N,Y
9,14.0,1,female,N,Y
10,4.0,1,female,N,Y


### Q3: 透過昨天課程的內容，驗證產生的兩個新變數，哪一個和目標變數(Survived_cate)的相關性較高?
* 提示:    
  首先觀察一下這些變數的資料型態後，再來想要以哪一種判斷倆倆的相關性。     

In [22]:
contTable_age = pd.crosstab(complete_data['Age_above65_'], complete_data['Survived_cate'])
contTable_age_female = pd.crosstab(complete_data['Age_above65_female'], complete_data['Survived_cate'])
display(contTable_age)
display(contTable_age_female)

Survived_cate,0,1
Age_above65_,,
N,414,289
Y,10,1


Survived_cate,0,1
Age_above65_female,,
N,350,92
Y,74,198


In [27]:
dof_age = min(contTable_age.shape[0], contTable_age.shape[1]) - 1
dof_age_female = min(contTable_age_female.shape[0], contTable_age.shape[1]) - 1
print(dof_age)
print(dof_age_female)

1
1


In [30]:
def judgment_CramerV(df, V):
    if V < 0.10:
        qual = 'negligible'
    elif V < 0.30:
        qual = 'small'
    elif V < 0.50:
        qual = 'medium'
    else:
        qual = 'large'
    return(qual)

In [25]:
# 離散 vs. 離散，使用 Cramer's V

crosstab, res = researchpy.crosstab(complete_data['Age_above65_'], complete_data['Survived_cate'], test='chi-square')
print("Cramer's value is", res.loc[2, 'results'])

Cramer's value is 0.0803


In [31]:
judgment_CramerV(dof_age, res.loc[2, 'results'])

'negligible'

In [32]:
crosstab, res = researchpy.crosstab(complete_data['Age_above65_female'], complete_data['Survived_cate'], test='chi-square')
print("Cramer's value is", res.loc[2, 'results'])

Cramer's value is 0.514


In [33]:
judgment_CramerV(dof_age, res.loc[2, 'results'])

'large'

* Age_above65_female 和 Survived_cate 有高度相關性